<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_API_TUTORIAL_BTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://docs.mistral.ai/platform/client/

https://github.com/mistralai/client-python

# Dependencies and Settings

In [ ]:
!nvidia-smi

Mon May 27 04:25:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   42C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Installing Modules

In [ ]:
!pip install colab-env --quiet
!pip install mistralai --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

Importing Modules

In [ ]:
import mistralai
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import os
import colab_env
import json

Mounted at /content/gdrive


# MISTRAL API SETTINGS

In [ ]:
api_key = os.environ["MISTRAL_API_KEY"]
client = MistralClient(api_key=api_key)

# Function Definitions

In [ ]:
def mistral_chat(prompt,model,client,stream):
  client = MistralClient(api_key=api_key)
  messages = [ChatMessage(role="user", content=prompt)]
  if stream==True:
     print("Streaming")
     print()
     print('Answer: ')
     for chunk in client.chat_stream(
        model=model,
        messages=messages,
        ):
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")
  else:

     #print("No streaming")
     chat_response = client.chat(model=model,messages=messages, safe_mode=True,temperature=0.9)
     #print(chat_response.choices[0].message.content)
     return chat_response

In [ ]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Summarize the below conversation."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['dialogue']}" if sample["dialogue"] else None
    response = f"{RESPONSE_KEY}\n{sample['summary']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

# Managenment

Model list


In [ ]:
from datetime import date
date.fromisoformat('2024-05-20')

datetime.date(2024, 5, 20)

https://docs.mistral.ai/getting-started/models/


open-mixtral-8x22b: A bigger sparse mixture of experts model. As such, it leverages up to 176B parameters but only uses about 39B during inference, leading to better inference throughput at the cost of more vRAM. Learn more on the dedicated blog post

{
  "object": "string",
  "data": [
    {
      "id": "string",
      "object": "string",
      "created": 0,
      "owned_by": "string"
    }
  ]
}

In [ ]:
model_list=client.list_models()
model_list.data[5].id

for model in model_list.data:
    print(model.id)
    #print(model.created)
    #print(model.owned_by)

open-mistral-7b
mistral-tiny-2312
mistral-tiny
open-mixtral-8x7b
open-mixtral-8x22b
open-mixtral-8x22b-2404
mistral-small-2312
mistral-small
mistral-small-2402
mistral-small-latest
mistral-medium-latest
mistral-medium-2312
mistral-medium
mistral-large-latest
mistral-large-2402
mistral-embed


https://mistral.ai/news/mixtral-8x22b/

In [ ]:
#model=model_list.data[11].id # mistral-large-latest
model=model_list.data[13].id
model

'mistral-large-latest'

In [ ]:
messages = [
    ChatMessage(role="user", content="What is the best French cheese?")
]

model=model_list.data[13].id

# No streaming
chat_response = client.chat(
    model=model,
    messages=messages,
)

print(chat_response.choices[0].message.content)

# BOB TRAVEL APP

In [ ]:
from IPython.display import Markdown
import textwrap
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
prompt= 'How do you plan out your trip? \
Bob is travelling to SAT from YVR \
1. He has a connection in DFW \
2. His connection is 6 hours long \
3. He has a budget of 100.00 including meals \
4. What can he do? Please suggest a time. \
5. Know- he is a hiker, museum, foodie, has a carry-on bag'

print(prompt)

response=mistral_chat(prompt,model,client,stream=False)

print('Answer: ')
#print(response.choices[0].message.content)
to_markdown(response.choices[0].message.content)

In [ ]:
print(response.choices[0].message.content)

Sure, I'd be happy to help Bob plan his trip! Here's a suggested itinerary:

1. **Preparation**: Bob should ensure he has all necessary travel documents, such as a valid ID or passport, and check-in online to save time at the airport. He should also pack his carry-on bag with essentials, including a change of clothes, travel-sized toiletries, and any necessary medications.

2. **Flight from YVR to DFW**: Bob should arrive at YVR at least 2 hours before his flight to allow time for security and any unexpected delays. During the flight, he can rest or enjoy in-flight entertainment.

3. **Layover at DFW**: Bob has a 6-hour layover at DFW. Here's a suggested breakdown:

   - **Meals (approx. $40)**: DFW has a variety of dining options. Bob can enjoy a meal at a sit-down restaurant like the Salt Lick BBQ or grab a quick bite at one of the many fast-food outlets.

   - **Explore the Airport (Free)**: DFW has several art installations and exhibits that Bob can visit for free. He can also take